# Canal plan 2D

In [ ]:
from trustutils import run 
from trustutils import visit
from trustutils.jupyter import plot
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import copy
import os, sys
from string import Template
import time
from IPython.display import display

run.introduction("C. Reiss","25/05/2022")
run.description(r'On envoie un liquide dans un canal plan 2D qui a toujours les memes dimensions () pour tester les lois de paroi adaptatives et leur sensibilite au maillage pour differentes discretisations de tau.')

run.TRUST_parameters()

In [ ]:
force_recalculation = True # if False, existing results will be used if the build directory exists, e.g. the trust-nk calculation will be skipped to quickly show results
number_of_partitions = 16
build_dir = run.BUILD_DIRECTORY

config_tau = ["int", "sup", "sup_lin"]
config_WL  = ["WR", "WL"]

calc_all = 0
n_calc   = 7
tab_calc = [ [ [ (i+1)%4 for i in range(n_calc)] ,
               [ (i+1)%2 for i in range(n_calc)] ] ,
             [ [ (i+1)%4 for i in range(n_calc)] ,
               [ (i+1)%4 for i in range(n_calc)] ] ,
             [ [ (i+1)%4 for i in range(n_calc)] ,
               [ (i+1)%2 for i in range(n_calc)] ]
           ]

title_raf  = [ 1 ,  2  ,  4  ,  8    ,  20  ,  40   ,  80    ,  120    ,  200 ]
y_0        = [.1 , .05 , .025, .0125 , .005 , .0025 , .00125 , .000833 , .0005]
y_1        = [.2 , .1  , .05 , .05   , .05  , .05   , .05    , .05     , .05  ]
ly_2       = [.2 , .1  , .15 , .15   , .15  , .15   , .15    , .15     , .15  ]
y_2        = np.array(y_1)+np.array(ly_2)
ly_3       = [.6 , .8  , .8  , .8    , .8   , .8    , .8     , .8      , .8   ]
y_3        = np.array(y_2)+np.array(ly_3)
nb_nodes_y1= [ 2 ,  2  ,  2  ,  3    ,  6   ,  11   ,  21    ,  31     ,  51  ]
nb_nodes_y2= [ 2 ,  2  ,  4  ,  7    ,  16  ,  16   ,  21    ,  31     ,  51  ]
nb_nodes_y3= [ 4 ,  9  ,  17 ,  17   ,  21  ,  21   ,  21    ,  31     ,  51  ]
lx         = 20
nb_nodes_x = [ 6 ,  11 ,  21 ,  25   ,  41  ,  46   ,  61    ,  91     , 151  ]
facsec     = [ 1 ,  2  ,  2  ,  2    ,  2   ,  2    ,  2     ,  2      ,  2   ]
u_0        = 1
rho        = 1e3
mu         = 2e-2
tmax       = 50

substitution_Wlu = ['paroi_fixe', 'paroi_frottante_loi { }']
substitution_Wlk = ['scalaire_impose_paroi Champ_Front_Uniforme 1 .0', 'cond_lim_k_simple']
substitution_diffusion_tau = ["tau", "turbulente", "turbulente"]
substitution_diffusion_supplementaire_tau = [" ",  ", Diffusion_supplementaire_echelle_temp_turb",", Diffusion_supplementaire_lin_echelle_temp_turb"]

def GenerateInputFile(name,substitutions_dict):
    with open(f"{build_dir}/jdd.data", "r") as file: 
        filedata = Template(file.read())
    result = filedata.substitute(substitutions_dict)
    with open(f"{build_dir}/{name}.data", "w") as file:
        file.write(result)


if force_recalculation or not os.path.exists(build_dir):
    print('Recalculation running..')
    run.reset()
    # Trigger the deletion of build directory and copy of all files inside the src directory into a new build directory:
    run.defaultSuite_ = run.TRUSTSuite(runPrepare=False)

    for y in range(n_calc) :
        print(f"Adding the launch files for a refinement of {title_raf[y]}")
        os.system(f'mkdir {build_dir}/{title_raf[y]}') 
        for wl in range(len(config_WL)) :
            for to in range(len(config_tau)):
                if (calc_all == 1) or (tab_calc[to][wl][y]==1) :
                    name = config_tau[to] + "_" + config_WL[wl] + "_raf_" + str(title_raf[y])
                    substitutions_dict = {"facsec" : str(facsec[y]),
                                        "WLu" : substitution_Wlu[wl],
                                        "WLk" : substitution_Wlk[wl],
                                        "diffusion_tau" : substitution_diffusion_tau[to],
                                        "diffusion_supplementaire_tau" : substitution_diffusion_supplementaire_tau[to],
                                        "y_1" : str(y_1[y]),
                                        "ly_2": str(ly_2[y]),
                                        "y_2" : str(y_2[y]),
                                        "ly_3": str(ly_3[y]),
                                        "y_3" : str(y_3[y]),
                                        "ny_1": str(nb_nodes_y1[y]),
                                        "ny_2": str(nb_nodes_y2[y]),
                                        "ny_3": str(nb_nodes_y3[y]),
                                        "lx"  : str(lx) ,
                                        "nx"  : str(nb_nodes_x[y]) ,
                                        "tmax": str(tmax),
                                        "rho" : str(rho),
                                        "mu"  : str(mu),
                                        "u_0" : str(u_0)
                                         }
                    GenerateInputFile(name,substitutions_dict)
                    if number_of_partitions == 1:
                        run.addCase(".", f"{name}.data")
                    else:
                        os.system(f'mv {build_dir}/{name}.data {build_dir}/{title_raf[y]} ; cd {build_dir}/{title_raf[y]};  make_PAR.data {name} {number_of_partitions}; cd ../..')
                        run.addCase(f"./{title_raf[y]}", f"PAR_{name}.data",nbProcs=number_of_partitions)
    run.runCases()
    display(run.tablePerf())

else:
    print('No recalculation, reusing old results!')

## A remplir

In [ ]:
nu = mu/rho
B_w = 0.075
B_k = 0.09
von_karman_ = 0.41
limiteur_y_p = 0.01


def u_plus_de_y_plus( y_p) :
    reichardt = np.log(1+0.4*y_p)/von_karman_
    reichardt += 7.8
    reichardt += -7.8*np.exp(-y_p/11)
    reichardt += -7.8*y_p/11*np.exp(-y_p/3)

    log_law = np.log(y_p)/von_karman_ + 5.1

    blending = np.tanh( y_p/27*y_p/27*y_p/27*y_p/27)

    return (1-blending)*reichardt + blending*log_law

def deriv_u_plus_de_y_plus(y_p) :
    reichardt = np.log(1+0.4*y_p)/von_karman_ + 7.8 -7.8*np.exp(-y_p/11) -7.8*y_p/11*np.exp(-y_p/3)
    log_law = np.log(y_p)/von_karman_ + 5.1
    blending = np.tanh( y_p/27*y_p/27*y_p/27*y_p/27)

    d_reichardt = 0.4/(1+0.4*y_p)*1/von_karman_
    d_reichardt += 7.8/11*np.exp(-y_p/11)
    d_reichardt += -7.8/11*np.exp(-y_p/3) + 7.8*y_p/33*np.exp(-y_p/3) 
    d_log_law = 1/(y_p)*1/von_karman_
    d_blending = 4/27*(y_p/27*y_p/27*y_p/27)*(1-blending*blending)

    return (1-blending)*d_reichardt - reichardt*d_blending + blending*d_log_law + d_blending*log_law 

def deriv_u_plus_de_y_plus_2(y_p) :
    reichardt = np.log(1+0.4*y_p)/von_karman_ + 7.8 -7.8*np.exp(-y_p/11) -7.8*y_p/11*np.exp(-y_p/3)
    log_law = np.log(y_p)/von_karman_ + 5.1
    blending = np.tanh( y_p/27*y_p/27*y_p/27*y_p/27)
    
    d_reichardt = 0.4/(1+0.4*y_p)*1/von_karman_
    d_reichardt += 7.8/11*np.exp(-y_p/11)
    d_reichardt += -7.8/11*np.exp(-y_p/3) + 7.8*y_p/33*np.exp(-y_p/3)
    d_log_law = 1/(y_p)*1/von_karman_
    d_blending = 4/27*(y_p/27*y_p/27*y_p/27)*(1-blending*blending)
    
    d_2_reichardt = -0.4*0.4/((1+0.4*y_p)*(1+0.4*y_p))*1/von_karman_
    d_2_reichardt += -7.8/(11*11)*np.exp(-y_p/11)
    d_2_reichardt += 7.8/33*np.exp(-y_p/3) + 7.8/33*np.exp(-y_p/3) - 7.8*y_p/99*np.exp(-y_p/3) 
    d_2_log_law = -1/((y_p)*(y_p))*1/von_karman_
    d_2_blending = 3*4/(27*27)*(y_p/27*y_p/27)*(1-blending*blending) + 4/27*(y_p/27*y_p/27*y_p/27)*(-2*d_blending*blending)
    
    rep = (1-blending)*d_2_reichardt-2*d_reichardt*d_blending-reichardt*d_2_blending
    rep += blending*d_2_log_law + 2*d_blending*d_log_law + d_2_blending*log_law
    return rep

def k_plus_de_y_plus(y_p) :
    return omega_plus_de_y_plus(y_p)*(1/deriv_u_plus_de_y_plus(y_p)-1)

def k_plus_de_y_plus_2(y_p) :
    arg = y_p/4
    blending = np.tanh(arg**4)

    return blending*omega_plus_de_y_plus(y_p)*(1/deriv_u_plus_de_y_plus(y_p)-1) + (1-blending)*5e-2*y_p **1.5

def tau_plus_de_y_plus(y_p) :
    return 1/omega_plus_de_y_plus(y_p)

def omega_plus_de_y_plus(y_p) :
    arg = y_p/10
    phi = np.tanh(arg**4)
    w_vis_p = 6/(B_w*y_p**2)
    w_log_p = 1/(B_k**0.5*von_karman_*y_p)
    w_1_p = w_vis_p+w_log_p
    w_2_p = (w_vis_p**1.2+w_log_p**1.2)**(1/1.2)
    return phi * w_1_p + (1-phi)*w_2_p

def theta_plus_de_y_plus(y_p) :
    Gamma = 0.01*(Pr*y_p)**4/(1+5*Pr**3*y_p)
    B = (3.85*Pr**(1/3)-1.3)**2+2.12*np.log(Pr)
    return Pr*y_p*np.exp(-Gamma) + (2.12*np.log(1+y_p)+B)*np.exp(-1/Gamma)


def calc_dyplus_kplus(y_p):
    arg = y_p/10
    phi = np.tanh(arg*arg*arg*arg)
    w_vis_p = 6/(B_w*y_p*y_p)
    w_log_p = 1/(np.sqrt(B_k)*von_karman_*y_p)
    w_1_p = w_vis_p+w_log_p
    w_2_p = (w_vis_p**1.2+w_log_p**1.2)**(1/1.2)
    w = phi * w_1_p + (1-phi)*w_2_p;
    
    d_w_vis = - 12 / (B_w * y_p*y_p*y_p)
    d_w_log = - 1 / ( np.sqrt(B_k) * von_karman_ * y_p*y_p)
    d_w_1 = d_w_vis + d_w_log 
    d_w_2 = ( d_w_vis * w_vis_p**0.2 + d_w_log * w_log_p**0.2)*( w_vis_p**1.2 + w_log_p**1.2)**(1/1.2-1 )
    d_phi = 4/10*(arg*arg*arg)*(1-phi*phi)
    d_w = d_phi * w_1_p + phi * d_w_1 - d_phi * w_2_p + (1-phi) * d_w_2 
    
    d_u_plus   = deriv_u_plus_de_y_plus(y_p)
    d_2_u_plus = deriv_u_plus_de_y_plus_2(y_p)
    return (1/d_u_plus-1)*d_w + w*(-d_2_u_plus)/(d_u_plus*d_u_plus)

def k_plus_de_y_plus_kalitzin(y_p) :

    f1 = (y_p-1) **3/1.8
    
    f2 = 3.26-.8*abs(2.24 - np.log(y_p))**3.2
    
    b = np.tanh( (y_p/2.5)**10)
    
    return np.maximum(f1*(1-b) + f2*b, 0)


img = plt.imread("k_th_kalitzin_2004.png")
fig = plt.figure(figsize = (10, 10))

abs_th = np.array([np.exp(i/20) for i in range(-20, 100)])
k_pl_th= k_plus_de_y_plus_kalitzin(abs_th)

plt.imshow(img, extent=[-1, 5, 0, 4], aspect=1)
plt.plot(np.log(abs_th), k_pl_th)
plt.xlim(-1, 5)
plt.ylim(-1,4)

In [ ]:
nom_config = ["int", "sup", "sup_2"]
nom_WL = ["WR", "WL", "symk"]
#facsec 50

dt_ev = [ [ [ -1 for y in range(n_calc) ] for wl in range(len(config_WL)) ] for to in range(len(config_tau))]
tab_name = copy.deepcopy(dt_ev)
ke  = copy.deepcopy(dt_ev)
tau = copy.deepcopy(ke)
vit = copy.deepcopy(ke)
yp_tr = copy.deepcopy(ke)
res = [vit, ke, tau, yp_tr]

name_phys = ["vit", "k", "tau"]
mark_WL  = [ "--", "-" ] 
color_raf = ["red", "hotpink", "magenta", "darkviolet", "blue", 'deepskyblue', 'cyan', "lime", "green", "olive", "gold", 'orange', "peru"]

for to in range(len(dt_ev)) :
    for wl in range(len(dt_ev[to])) : 
        for y in range(n_calc) :
            if (calc_all == 1) or (tab_calc[to][wl][y]==1) :
                name = config_tau[to] + "_" + config_WL[wl] + "_raf_" + str(title_raf[y])
#                print("loading " + name + " results...")
                tab_name[to][wl][y] = name
                dt_ev[to][wl][y] = pd.read_csv(f"{build_dir}/{title_raf[y]}/PAR_"+name+".dt_ev", "\t")
                dt_ev[to][wl][y] = dt_ev[to][wl][y].iloc[:,:10]
                dt_ev[to][wl][y].columns = ['time', 'dt', 'facsec', 'residu', 'dt_stab', 'vit', 'alpha', 'T', 'tau', 'k']

                res[0][to][wl][y] = np.array(plot.loadText(f"{title_raf[y]}/PAR_{name}_VIT.son")[1::2,1])
                res[1][to][wl][y] = np.array(plot.loadText(f"{title_raf[y]}/PAR_{name}_K.son")[1::,1])
                res[2][to][wl][y] = np.array(plot.loadText(f"{title_raf[y]}/PAR_{name}_TAU.son")[1::,1])
                res[3][to][wl][y] = plot.loadText(f"{build_dir}/{title_raf[y]}/PAR_{name}_Y_PLUS.son")[1,1]

#                print(name + " results successfully loaded !")


In [ ]:
matplotlib.rcParams.update({'font.size': 14})

plt.figure(figsize = (20,20))

for l in range(3): #phys
    for i in range(len(dt_ev)) : #tau
        plt.subplot(int("33"+str(1+l + 3*i)))
        for j in range(len(dt_ev[i])) : # WL
            for k in range(len(dt_ev[i][j])) : # raf
                if (calc_all == 1) or (tab_calc[i][j][k]==1) :
                    plt.plot(dt_ev[i][j][k]["time"][::], dt_ev[i][j][k][name_phys[l]][::], mark_WL[j], label = tab_name[i][j][k], color = color_raf[k])

        plt.xlim(0,50)
        plt.ylim(1e-6, 1e1)
        plt.yscale("log")
        plt.title("dt_ev "+name_phys[l] + " tau " + config_tau[i])
        plt.xlabel('time')
        plt.ylabel("dt_ev " + name_phys[l])
        plt.legend()

        plt.tight_layout()

In [ ]:
y = np.linspace(0.00025, 0.99975 ,1000)
y_min = [0 for i in range(len(y_0))]
y_pas = [0 for i in range(len(y_0))]
for i in range(len(y_0)) :
    y_min[i] = int( (y_0[i]+1e-8) * 1000)
    y_pas[i] = int( (y_1[i]+1e-8) * 1000 / (nb_nodes_y1[i]-1))

tab_res = [vit, ke, tau]

plt.figure(figsize = (20,20))

scale = ["log", "linear"]

for l in range(3): #phys
    for i in range(len(dt_ev)) : #tau
        plt.subplot(int("33"+str(1+l + 3*i)))
        for j in range(len(dt_ev[i])) : # WL
            for k in range(len(dt_ev[i][j])) : # raf
                if (calc_all == 1) or (tab_calc[i][j][k]==1) :
                    plt.plot(y[y_min[k]::y_pas[k]], res[l][i][j][k][y_min[k]::y_pas[k]], mark_WL[j], label = tab_name[i][j][k], color = color_raf[k])
    
        plt.title(f"{name_phys[l]} in {config_tau[i]} configuration for tau diffusion")
        plt.xlabel("distance to wall (m)")
        plt.ylabel(name_phys[l])
        plt.legend()


In [ ]:
u_tau_mano = .047

plt.figure(figsize = (8,8))

plt.subplot(211)

j = 1 # on ne regarde que les cas avec loi de paroi
for i in range(len(dt_ev)) :
    for k in range(n_calc) :
        if (calc_all == 1) or (tab_calc[i][j][k]==1) :
            if k == 0 :
                plt.plot(y_0[k], res[3][i][j][k], "o", color = color_raf[3*i], label = config_tau[i])
            else : 
                plt.plot(y_0[k], res[3][i][j][k], "o", color = color_raf[3*i])


plt.xlim(2e-4, 2e-1)
plt.xlabel(r"y in the first element (m)")
plt.ylabel("y_p calculated by TRUST")
plt.yscale("log")
plt.xscale("log")
plt.legend()

plt.subplot(212)

u_tau = 0
n_calc_u_tau = 0

j = 1 # on ne regarde que les cas avec loi de paroi
for i in range(len(dt_ev)) :
    for k in range(n_calc) :
        if (calc_all == 1) or (tab_calc[i][j][k]==1) :
            if k == 0 :
                plt.plot(y_0[k], res[3][i][j][k]*nu/y_0[k], "o", color = color_raf[3*i], label = config_tau[i])
            else : 
                plt.plot(y_0[k], res[3][i][j][k]*nu/y_0[k], "o", color = color_raf[3*i])
            u_tau += res[3][i][j][k]*nu/y_0[k]
            n_calc_u_tau+=1

u_tau /= n_calc_u_tau

plt.xlim(2e-4, 2e-1)
plt.xlabel(r"y in the first element (m)")
plt.ylabel(r"$u_\tau$ calculated by TRUST")
plt.yscale("log")
plt.xscale("log")
plt.plot([2e-4, 2e-1], [u_tau, u_tau], 'k--', label = "Average")
plt.legend()

plt.tight_layout()

y_p = np.array(y) * u_tau/nu
y_p_0 = np.array(y_0) * u_tau/nu
tab_res_p = copy.deepcopy(tab_res)
tab_th = [u_plus_de_y_plus(y_p), k_plus_de_y_plus_kalitzin(y_p), tau_plus_de_y_plus(y_p)]
for i in range(len(dt_ev)) : #tau
    for j in range(len(dt_ev[i])) : # WL
        for k in range(len(dt_ev[i][j])) : # raf
            tab_res_p[0][i][j][k] /= u_tau;
            tab_res_p[1][i][j][k] /= u_tau**2;
            tab_res_p[2][i][j][k] *= u_tau**2/nu;
            

In [ ]:
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

fig = plt.figure(figsize = (20,20))

tab_loc = [4 , 1 , 4]
tab_size_inset = ["60%", "60%", "52%"]
y_ax_name = [r"$u_+$", r"$k_+$", r"$\tau_+$"]
inset_scale = ["log", "log", "linear"]

for l in range(3): #phys
    for i in range(len(dt_ev)) : #tau
        ax = fig.add_subplot(int("33"+str(1+l + 3*i)))
        for j in range(len(dt_ev[i])) : # WL
            for k in range(len(dt_ev[i][j])) : # raf
                if (calc_all == 1) or (tab_calc[i][j][k]==1) :
                    ax.plot(y_p[y_min[k]::y_pas[k]], tab_res_p[l][i][j][k][y_min[k]::y_pas[k]], mark_WL[j], label = tab_name[i][j][k], color = color_raf[k])
        
        ax.plot(y_p, tab_th[l%3], "k",  label = "theory")
        ax.set_xscale("linear")
        ax.set_title(f"{name_phys[l]} in {config_tau[i]} configuration for tau diffusion")
        ax.set_xlabel(r"$y_+$")
        ax.set_ylabel(y_ax_name[l])
        
        if l==1 :
            ax.set_ylim(0,14)

        if l != 3 :
            inset_ax = inset_axes(ax, 
                        width=tab_size_inset[l], # width = 30% of parent_bbox
                        height=tab_size_inset[l], # height : 1 inch
                        loc=tab_loc[l])

            for j in range(len(dt_ev[i])) : # WL
                for k in range(len(dt_ev[i][j])) : # raf
                    if (calc_all == 1) or (tab_calc[i][j][k]==1) :
                        inset_ax.plot(y_p[y_min[k]::y_pas[k]], tab_res_p[l][i][j][k][y_min[k]::y_pas[k]], mark_WL[j], label = tab_name[i][j][k], color = color_raf[k])

            inset_ax.plot(y_p, tab_th[l%3], "k",  label = "BL theory")
            inset_ax.set_xscale(inset_scale[l])
            inset_ax.set_yticks([])
            inset_ax.tick_params(axis="x",direction="in", pad=-20)
            if l==1 :
                inset_ax.set_ylim(-.5, 5)
            if l==2 :
                plt.xlim(-10,220)
                plt.ylim(0, 25)
    
                
        if l==2 :
            ax.legend(prop={'size': 10})
            ax.set_ylim(-5, 200)
            
plt.tight_layout()

In [ ]:
# y_0 pour differents raffinements

fig = plt.figure(figsize = (6,4))

y_p_0 = np.array(y_0) * u_tau / nu

plt.plot(title_raf, y_0, 'o', label = "$y_+$")
plt.xscale('log')
plt.yscale('log')

for i in range(len(y_0)):
    plt.annotate(str(y_p_0[i])[:4],xy=(title_raf[i]*1.2,y_0[i]))

plt.xlim(.5, 1000)
plt.ylim(2e-4, .2)
plt.legend()
plt.title(r"Range of $y_+$ that can be used in this validation sheet")
plt.xlabel("Refinement")
plt.ylabel("y in the first element (m)")
plt.tight_layout()
plt.show()

In [ ]:
# temps de calcul pour differentes diffusions
time = copy.deepcopy(tab_calc)

plt.figure(figsize = (8,8))

plt.subplot(211)

j = 1 # on ne regarde que les cas avec loi de paroi
for i in range(len(dt_ev)) : #tau
    for k in range(len(dt_ev[i][j])) : # raf
        if (calc_all == 1) or (tab_calc[i][j][k]==1) :
            f = os.open(f"{build_dir}/{title_raf[k]}/PAR_{tab_name[i][j][k]}.perf", os.O_RDONLY)
            time[i][j][k] = float(os.read(f, 200).split()[3])
            plt.plot(y_p_0[k], time[i][j][k], 'o', label = tab_name[i][j][k], color = color_raf[3*i])

plt.xlim(.5, 1000)
plt.xlabel(r"$y_+$ in the first element (m)")
plt.ylabel("Calculation time")
plt.yscale("log")
plt.xscale("log")
plt.legend(prop={'size': 10})

plt.subplot(212)

for i in range(len(dt_ev)) : #tau
    for k in range(len(dt_ev[i][j])) : # raf
        if (calc_all == 1) or (tab_calc[i][j][k]==1) :
            if k == 0 :
                plt.plot(y_p_0[k], time[i][j][k]/time[0][j][k], 'o', label = config_tau[i], color = color_raf[3*i])
            else : 
                plt.plot(y_p_0[k], time[i][j][k]/time[0][j][k], 'o', color = color_raf[3*i])


plt.xlim(.5, 1000)
plt.xlabel("y in the first element (m)")
plt.ylabel("Calc time/integrated diffusion")
plt.xscale("log")


plt.tight_layout()